This program will be an example to showcase how to use the edc522, keithley193a (both non SCPI instruments) as well as the arduino model and showcase how to make a rudimentary program

In [1]:
from piec.drivers.utilities import PiecManager
from piec.drivers.arduino import Arduino_Stepper
from piec.drivers.edc522 import EDC522
from piec.drivers.keithley193a import Keithley193a
pm = PiecManager()


In [2]:
pm.list_resources()

('GPIB0::9::INSTR',)

In [3]:
Stepper = Arduino_Stepper("ASRL3::INSTR")
Stepper.idn()

VisaIOError: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.

In [5]:
voltage_program = EDC522("GPIB0::9::INSTR")
voltage_program.idn()

'ID?\r\n001'

In [ ]:
def set_output(self, value, mode="voltage"):
        """
        Formats a current or voltage value into an 8-character string for instrument control.
        Automatically determines the appropriate range.

        Args:
            value (float or int): The value to send to the instrument.
            mode (str, optional): "voltage" or "current". Defaults to "voltage".

        Returns:
            str: An 8-character command string, or None if input is invalid or value is out of range.
        """

        if mode not in ("voltage", "current"):
            return None

        if value == 0:
            return "00000000"

        polarity = "+" if value > 0 else "-"
        abs_value = abs(value)

        if mode == "voltage":
            ranges = [0.1, 10, 100, 1000]
            range_chars = "0123"
            max_values = [0.9999999, 10, 100, 1000]  # Slightly higher max values
        elif mode == "current":
            ranges = [0.01, 0.1]
            range_chars = "45"
            max_values = [0.00999999, 0.1]  # Slightly higher max values
        else:
            return None

        for i, r in enumerate(ranges):
            if abs_value <= max_values[i]:  # Check against max value for the range
                scaled = abs_value / r
                best_range_index = i
                scaled_value = scaled
                break
        else:  # No suitable range was found
            return None

        digits_str = "{:06.0f}".format(scaled_value * 1000000)

        # J handling:
        if best_range_index == 1 and scaled_value == 10:  # Exactly 10V
            digits_str = "J00000"
        elif best_range_index == 1 and 1 <= scaled_value < 10: # 1V to 9.99999V in 10V range
            digits = []
            for digit in str(int(scaled_value * 1000000)):
                if digit == '1':
                    digits.append('J')
                else:
                    digits.append(digit)
            digits_str = "".join(digits).zfill(6)
        command = f"{polarity}{digits_str}{range_chars[best_range_index]}"
        print(command)

In [11]:
set_output('bet', 1)

+0100001


In [8]:
voltage_program.instrument.write("+1000001")

10

In [9]:
voltage_program.set_output(1)

In [ ]:
voltage_out = Keithley193a()
voltage_out.idn()   # This will print the IDN of the instrument